In [1]:
from Program import Program, Instruction
import Selection
import random
import pickle
import math
MIN_LENGTH = 1
MAX_LENGTH = 256
Selection.set_max_length(MAX_LENGTH)
CASES = []
Selection.MAX_LENGTH

256

In [2]:
def rand():
    """ Generates a Random Program """
    random_program = Program(IS=['Add', 'Sub', 'Mul', 'Div', 'Sin', 'Mean', 'Copy', \
                                 'Sqrt', 'Sqr', 'Max', 'Min', 'Exp', 'Log', 'Lt', 'Gte', \
                                 'Eq', 'Neq', 'And', 'Or', 'Not', 'If'])
    return random_program

def I(program, inp):
    """ Runs 'program' on input inp. Clips the return value to lie in [-1,1] as per the HW instructions """
    program.reset()
    program._set_input(inp)
    return_val = program.execute()
    # if return_val > 1:
    #     return 1
    # if return_val < -1:
    #     return -1
    return return_val

def f(program, gen, cases=CASES):
    """ Returns the sum of the squared error on the fitness cases for evaluating sin """
    F1_AMELIORATION = 2_500
    fitness = 0
    dif     = 0
    outs    = []
    fitness_1 = 0
    fitness_2 = 0
    out_counts = {}
    for i in range(len(cases)):
        tv = math.sin(cases[i])     #true value of sin for the fitness case
        rv = I(program, cases[i])   #return value of the program evaluated on the fitness case
        outs.append(rv) 
        fitness_1 += math.sqrt((rv-tv)**2)
        for out in outs:
            if out in out_counts.keys():
                out_counts[out] += 1
            else:
                out_counts[out] = 1
        for out_count in out_counts.keys():
            if out_counts[out_count] > 1:
                fitness_2 += out_counts[out_count]**2
    fitness = max([(1-(gen/F1_AMELIORATION)), 0])*fitness_2 + min([(gen/F1_AMELIORATION), 1])*fitness_1 
    return fitness

def f1(program, cases=CASES):
    F1_AMELIORATION = 2_500
    fitness = 0
    dif     = 0
    outs    = []
    fitness_1 = 0
    fitness_2 = 0
    out_counts = {}
    for i in range(len(cases)):
        tv = math.sin(cases[i])     #true value of sin for the fitness case
        rv = I(program, cases[i])   #return value of the program evaluated on the fitness case
        outs.append(rv) 
        fitness_1 += math.sqrt((rv-tv)**2)
    
    #     for out in outs:
    #         if out in out_counts.keys():
    #             out_counts[out] += 1
    #         else:
    #             out_counts[out] = 1
    #     for out_count in out_counts.keys():
    #         if out_counts[out_count] > 1:
    #             fitness_2 += out_counts[out_count]**2
    # fitness = max([(1-(gen/F1_AMELIORATION)), 0])*fitness_2 + min([(gen/F1_AMELIORATION), 1])*fitness_1 
    return fitness_1
    
    
def f2(program, generation=None, cases=CASES):
    F1_AMELIORATION = 2_500
    fitness = 0
    dif     = 0
    outs    = []
    fitness_1 = 0
    fitness_2 = 0
    out_counts = {}
    for i in range(len(cases)):
        #tv = math.sin(cases[i])     #true value of sin for the fitness case
        rv = I(program, cases[i])   #return value of the program evaluated on the fitness case
        outs.append(rv) 
        #fitness_1 += math.sqrt((rv-tv)**2)
    for out in outs:
        if out in out_counts.keys():
            out_counts[out] += 1
        else:
            out_counts[out] = 1
    for out_count in out_counts.keys():
        if out_counts[out_count] > 1:
            fitness_2 += out_counts[out_count]**2
    # fitness = max([(1-(gen/F1_AMELIORATION)), 0])*fitness_2 + min([(gen/F1_AMELIORATION), 1])*fitness_1 
    return fitness_2

def mut(program):
    #PROBABILITIES TO
    ADD_P = 0.15   #Add a random instruction in a random location
    REM_P = 0.1   #Delete a random instruction
    NAM_P = 0.1   #Change the name of an instruction (and therefore its type)
    ARG_P = 0.1   #Change an argument of an instruction
    SWP_P = 0.1   #Swap two instructions
    
    if random.random() < ADD_P:  #Add instruction
        if (len(program.INST) < MAX_LENGTH):
            rand_inst = Instruction(len(program.WREG), len(program.CREG), instruction_set=program.IS)
            loc       = random.randint(0, len(program.INST))
            program.INST.insert(loc, rand_inst)
            
    if random.random() < REM_P:  #Remove instruction
        if (len(program.INST) > MIN_LENGTH):
            loc       = random.randint(0, len(program.INST)-1)
            program.INST.pop(loc)
            
    if random.random() < NAM_P:  #Change an instruction name
        if(len(program.INST) > 1):
            loc = random.randint(0, len(program.INST)-1)
            program.INST[loc]._set_name(random.choice(program.IS))
        
    if random.random() < ARG_P:   #Change argument to an instruction
        if(len(program.INST) > 1):
            inst_loc = random.randint(0, len(program.INST)-1)
            arg_loc  = random.randint(0,2)
            if arg_loc == 1:
                program.INST[inst_loc]._set_op1(random.randint(0, len(program.REG)))
            if arg_loc == 2:
                program.INST[inst_loc]._set_op2(random.randint(0, len(program.REG)))
            if arg_loc == 0:
                program.INST[inst_loc]._set_dest(random.randint(0, len(program.WREG)))
            
    if random.random() < SWP_P:   #Swap the index of two instructions
        if (len(program.INST) > 1):
            loc1 = random.randint(0, len(program.INST)-1)
            loc2 = random.randint(0, len(program.INST)-1)
            temp = program.INST[loc1]
            program.INST[loc1] = program.INST[loc2]
            program.INST[loc2] = temp
            
def XOver(prog1, prog2):
    """ Crossover Function chooses 2 random crossover points (one for each individual)
        Swaps the second part of program 2's instructions with the first part of program 1's instructions """
    #TODO: Make sure Xover cannot create programs that are too long!
    #Maybe determine the length of each program after loc1 and loc2 are selected and modify if one would be too long
    length_after_Xover_prog1 = MAX_LENGTH + 1
    length_after_Xover_prog2 = MAX_LENGTH + 1
    
    if ((len(prog1.INST) > 1) and (len(prog2.INST) > 1)):
        while(length_after_Xover_prog1 > MAX_LENGTH or length_after_Xover_prog2 > MAX_LENGTH):
            loc1 = random.randint(0, len(prog1.INST)-1)
            loc2 = random.randint(0, len(prog2.INST)-1)
            temp1 = prog1.INST[:loc1]
            temp2 = prog2.INST[loc2:]
            length_after_Xover_prog1 = len(temp2 + prog1.INST[loc1:])
            length_after_Xover_prog2 = len(prog2.INST[:loc2] + temp1)
        prog1.INST = temp2 + prog1.INST[loc1:]
        prog2.INST = prog2.INST[:loc2] + temp1
    

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('data_banknote_authentication.csv')
target = data.iloc[:,-1]
data   = data.iloc[:,:-1]
X_train, X_test1, y_train, y_test1 = train_test_split(data, target, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test       = train_test_split(X_test1, y_test1, test_size=0.5, random_state=42)

In [4]:
from sklearn.metrics import roc_auc_score
def f3(program, cases=X_train.values, labels=y_train.values):
    outs = []
    for row in range(len(cases)):
        tv = labels[row]           #true value of sin for the fitness case
        #print(program)
        program.reset()             #return value of the program evaluated on the fitness case
        program._set_inputs(cases[row])
        #print(program)
        rv = program.execute()
        outs.append(rv) 
    if (max(outs) - min(outs) != 0):
        outs = [(float(i)-min(outs))/(max(outs)-min(outs)) for i in outs]
    elif max(outs) != 0:
        outs = [(float(i))/(max(outs)) for i in outs]
    fitness1 = roc_auc_score(labels, outs)

    return fitness1

In [ ]:
POP_SIZE       = 1250
XOVER_P        = 0.009
ELITE_K        = int(POP_SIZE*0.1)#50
population     = [Program(reg_init='z', IS=['Add', 'Sub', 'Mul', 'Div', 'Sin', 'Mean', 'Copy', \
                                 'Sqrt', 'Sqr', 'Max', 'Min', 'Exp', 'Log', 'Lt', 'Gte', \
                                 'Eq', 'Neq', 'And', 'Or', 'Not', 'If']) for i in range(POP_SIZE)] # Start with a population of 500 random programs
best_fitness_c = -1                   # Start this very high since even a random program will beat it
best_prog_c    = None                       # Keep track of the best program
tolerance      = 1                          # Stop searching when the best program has a fitness less than 1 (we want to minimize error)
generation     = 1

best_fitnesses_generation_c = []
avg_fitnesses_generation_c  = []
validation_fitnesses        = []
ten_best_validation         = []
max_validation              = 0
patience                    = 0
max_patience                = 100
CHECKPOINT                  = False
#TODO: Consider a fitness function that encourages putting DIFFERENT functions of the input in different registers. 10_000
#      is too big for a population and XOVER and mutation probabilities should be lowered.



logfile = open('BanknoteClassificationLog.txt', 'w+')

while (generation < 5000):

    print('Generation: ', generation)
    this_gen_fitness_c= []
    for individual in population:
        ind_fit = f3(individual)
        this_gen_fitness_c.append(ind_fit)
        if ind_fit > best_fitness_c:
            print('----------NEW BEST FITNESS---------------')
            print(ind_fit)
            print(individual)
            print('---------- EFFECTIVE PROGRAM ---------------')
            individual.print_effective_program()
            best_fitness_c = ind_fit
            best_prog_c    = individual
    best_fitnesses_generation_c.append(max(this_gen_fitness_c))
    avg_fitnesses_generation_c.append(sum(this_gen_fitness_c)/len(this_gen_fitness_c))
    
    ten_best = Selection.elite_selection(population, f3, X_train.values, pop_fitnesses=this_gen_fitness_c, k=10, opt_max=True)
    validation_fitnesses.append(f3(ten_best[0], X_val.values, y_val.values))
    val = [f3(ten_best[i], X_val.values, y_val.values) for i in range(len(ten_best))]
    avg_val = sum(val)/len(val)
    if avg_val > max_validation:
        dumpfile='./saved_generations/generationBest.pkl'
        with open(dumpfile, 'wb+') as du:
            pickle.dump(population, du)
    ten_best_validation.append(avg_val)
    if generation > 2 and ten_best_validation[-1] <= ten_best_validation[-2]:
        patience += 1
        CHECKPOINT = True
    else:
        patience = 0
        CHECKPOINT = False
    if CHECKPOINT:
        dumpfile='./saved_generations/generation' + str(generation) + '.pkl'
        with open(dumpfile, 'wb+') as du:
            pickle.dump(population, du)
    if patience > max_patience:
        print('Early Stopping Triggered at generation ', generation)
        print('Best Population saved at ', './saved_generations/generationBest.pkl')
        print('Checkpoints available by generation')
        
    logfile.write('-----------------GENERATION ' + str(generation) + '---------------------------\n')
    logfile.write("Fitnesses: [" + ", ".join(str(item) for item in this_gen_fitness_c) + "]\n")
    logfile.write('best fitness: ' +  str(best_fitnesses_generation_c[-1]) + "\n")
    logfile.write('avg fitnesses: ' +  str(avg_fitnesses_generation_c[-1]) + "\n")
    print('Best Fitness: ', best_fitnesses_generation_c[-1])
    print('Avg  Fitness: ', avg_fitnesses_generation_c[-1])
    print('Avg of 10 best validation fitness: ', ten_best_validation[-1])
    print('10 best validation fitness: ', val)
    elite = Selection.elite_selection(population, f3, X_train.values, pop_fitnesses=this_gen_fitness_c, k=int(POP_SIZE/10), opt_max=True)
    tournament = Selection.tournament_selection(population, f3, X_train.values, pop_fitnesses=this_gen_fitness_c, next_gen_size=int((POP_SIZE*8)/10), opt_max=True)
    randos     = [Program(reg_init='z',IS=['Add', 'Sub', 'Mul', 'Div', 'Sin', 'Mean', 'Copy', \
                                 'Sqrt', 'Sqr', 'Max', 'Min', 'Exp', 'Log', 'Lt', 'Gte', \
                                 'Eq', 'Neq', 'And', 'Or', 'Not', 'If']) for i in range(int(POP_SIZE/10))]
    # while(len(next_gen) < POP_SIZE-(POP_SIZE/10)): #Fill the next gen with tournament selection up to 90%
    #     r1 = random.randint(0, POP_SIZE-1)
    #     r2 = random.randint(0, POP_SIZE-1)
    #     if this_gen_fitness[r1] < this_gen_fitness[r2]:
    #         next_gen.append(population[r1]._clone())
    #     else:
    #         next_gen.append(population[r2]._clone())
            
    # while(len(next_gen) < POP_SIZE):              #Fill the rest of the population with random programs
    #     next_gen.append(Program(reg_init='z'))
    next_gen=elite+tournament+randos
    for program in range(len(next_gen)):
        mut(next_gen[program])
        if random.random() < XOVER_P:
            program2 = random.randint(0, len(next_gen)-1)
            XOver(next_gen[program], next_gen[program2])
            logfile.write('Crossed ' + str(program) + ' and ' + str(program2) +'\n')
    generation += 1
    population = next_gen
    for program in population:
        program.reset(new_gen=True)
    
            
logfile.close()

Generation:  1
----------NEW BEST FITNESS---------------
0.5
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 1.0, 0.5, 0.3333333333333333, 0.25, 0.2, -2.3629, -0.10554, 1.9336, 1.1358]
Sqrt(28,12,7)
Max(11,17,15)
Neq(30,28,9)
Gte(11,9,5)
Sub(23,17,17)
Log(26,7,4)
Add(15,5,8)
Lt(24,10,2)
Min(3,3,13)
Div(26,28,11)
Lt(22,26,16)
Lt(15,5,22)
Mul(29,10,16)

---------- EFFECTIVE PROGRAM ---------------
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 1.0, 0.5, 0.3333333333333333, 0.25, 0.2, -2.3629, -0.10554, 1.9336, 1.1358]

----------NEW BEST FITNESS---------------
0.7967468141948602
[5.583296409999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -2.3629, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 1.0, 0.5, 0.3333333333333333, 0.25, 0.2, -2.3629, -0.10554, 1.9336, 1.1358]
Eq(2,3,12)
Min(30,4,10)
Sub(30,26,22

In [5]:
p = rand()

In [6]:
p

[0.9030087848336675, 0.8178565575406394, 0.7818009656837759, 0.4658132359616881, 0.03192004981411478, 0.4727687801278482, 0.439872218968805, 0.8542784429318453, 0.16911837056370516, 0.6712132828041734, 0.5685071806676667, 0.5918724647269676, 0.050613642090601485, 0.9243455275216881, 0.2635673908700801, 0.491276441335618, 0.5221914394349015, 0.5405917936633469, 0.9351309023973787, 0.42141048345231846, 0.6474528517126301, 0.7336224262262239, 0.6578182354210087, 0.5800547990339724, -1.0, 1.0, 0.5, 0.3333333333333333, 0.25, 0.2, 0.0]
Exp(29,9,0)
Log(4,27,18)
Mul(4,15,4)
Log(5,6,21)
Eq(14,14,18)
Min(3,18,6)
Or(13,11,23)
And(17,11,6)
Min(14,24,13)
Eq(28,6,0)

In [7]:
ec = p.get_effective_code()

In [8]:
ec

[Mul(4,15,4), Min(3,18,6), And(17,11,6), Eq(28,6,0)]

In [7]:
X_train.values[0]

array([ 5.7227 ,  5.8312 , -2.4097 , -0.24527])

In [1]:
size = 5
l = [i for i in range(10)]
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [2]:
l[:size]

[0, 1, 2, 3, 4]

In [3]:
len(l[:size]) == size

True

In [2]:
from importlib import reload
reload(Selection)

<module 'Selection' from 'E:\\GPHw2\\Selection.py'>

In [9]:
X_val

,3.6216,8.6661,-2.8073,-0.44699
630,1.89940,0.97462,4.226500,0.813770
1070,-2.53730,-6.95900,8.805400,1.528900
1291,-0.76794,3.45980,-3.440500,-3.427600
824,-2.23400,-7.03140,7.493600,0.613340
377,2.85230,9.00960,-3.761000,-3.337100
...,...,...,...,...
358,-2.32420,11.51760,1.823100,-5.375000
163,4.88510,1.59950,-0.000291,1.640100
549,3.97190,1.03670,0.759730,1.001300
1081,-1.39460,2.31340,-0.444990,-1.490500


In [11]:
X_val.shape

(137, 4)

In [12]:
y_val.shape

(137,)